# Proyecto Primer Bimestre - Reuters
### Integrantes
- Carlos Córdova
- Hernán Sánchez
- Galo Tarapués

## Introducción 

El presente proyecto implementa un **sistema de recuperación de información** , diseñado para trabajar con el corpus *Reuters-21578*. 

El objetivo principal del proyecto es **es diseñar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578**. 

## Fases del proyecto 

El sistema se divide en varias fases interconectadas. Estas incluyen:

1. **Adquisición de los Datos**
2. **Preprocesamiento del corpus**
3. **Representación de Datos en Espacio Vectorial**
4. **Indexación**
5. **Mecanismo de búsqueda**
6. **Evaluación del sistema**

Los cuales seran detallados a continuación: 

### 1. **Adquisición de los Datos**

1. Carga de documentos: 

Los documentos se cargan desde una carpeta especifica del directorio del proyecto, que contiene noticias etiquetadas con categorías.

In [ ]:
import os
import logging
from typing import List, Dict

# Configuración de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_documents(directory_path: str) -> List[Dict]:
    
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.sgm'):  # Aseguramos que solo se procesen archivos SGML
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                    documents.append({'file_name': file, 'content': content})  # Guardamos el contenido
                except Exception as e:
                    logger.error(f"Error leyendo {file_path}: {e}")
    return documents


### 2. Preprocesamiento del corpus 

1. Librerias usadas 

In [ ]:
import os # Permite operaciones con rutas y archivos del sistema.
import re # Facilita la manipulación y limpieza de texto mediante expresiones regulares.
import nltk # Ofrece herramientas avanzadas de NLP como tokenización y manejo de stopwords.
import pandas as pd # Estructura los datos en un formato tabular para facilitar el análisis.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import logging # Registra eventos y errores para depuración y análisis posterior.

# Configuración de logs
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


2. Extracción de Contenido Relevante

Cada documento es convertido en un diccionario con los siguientes campos:

- id: Identificador único del documento (nombre del archivo).
- title: Título procesado.
- body: Cuerpo del texto procesado.
- categories: Lista de etiquetas asociadas al documento.

In [ ]:
def process_document(self, file_path: str) -> Dict:
    """
    Procesa un documento en bruto y extrae los campos relevantes.
    Args:
        file_path (str): Ruta del archivo del documento.
    Returns:
        Dict: Diccionario con 'id', 'title', 'body' y 'categories'.
    """
    try:
        # Leer el documento
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            content = f.read()
        
        # Separar título y cuerpo
        parts = content.split("\n", 1)
        title = parts[0] if len(parts) > 0 else ""
        body = parts[1] if len(parts) > 1 else ""
        
        # Obtener nombre del archivo como ID del documento
        doc_id = os.path.basename(file_path)
        
        # Asociar categorías desde el archivo externo
        categories = self.categories.get(doc_id, [])
        
        return {
            'id': doc_id,
            'title': title.strip(),
            'body': body.strip(),
            'categories': categories
        }
    except Exception as e:
        logger.error(f"Error procesando documento {file_path}: {str(e)}")
        return None


3. Limpieza del texto

El texto obtenido se limpia para eliminar caracteres no deseados, etiquetas HTML, y otros elementos residuales.

In [ ]:
import re

def clean_text(text: str) -> str:
    """
    Limpia el texto eliminando etiquetas HTML, caracteres no alfabéticos y normalizando.
    Args:
        text (str): Texto original.
    Returns:
        str: Texto limpio.
    """
    text = re.sub(r'<[^>]+>', '', text)  # Elimina etiquetas HTML
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Sustituye caracteres no alfabéticos
    text = text.lower()  # Convierte a minúsculas
    return text


4. Tokenización

- La función tokenize_text utiliza la función word_tokenize para dividir el texto limpio en tokens.
- Los tokens son devueltos como una lista de cadenas.

In [ ]:
from nltk.tokenize import word_tokenize

def tokenize_text(text: str) -> list:
    """
    Divide el texto en palabras individuales.
    Args:
        text (str): Texto limpio.
    Returns:
        list: Lista de palabras (tokens).
    """
    return word_tokenize(text)


5. Filtrado de stopwords

Las palabras stopwords son eliminadas en lo siguientes pasos:

- Primero se descarga el conjunto de stopwords en inglés de la librería nltk.
- La función remove_stopwords toma una lista de tokens y se compara con la lista de stopwords y se descarta si coincide.
- Devuelve una lista de tokens filtrada.

In [ ]:
from nltk.corpus import stopwords

def remove_stopwords(tokens: list, stopword_path: str) -> list:
    """
    Filtra palabras vacías de los tokens.
    Args:
        tokens (list): Lista de palabras tokenizadas.
        stopword_path (str): Ruta al archivo de stopwords personalizadas.
    Returns:
        list: Lista de tokens sin stopwords.
    """
    with open(stopword_path, 'r') as f:
        custom_stopwords = set(f.read().splitlines())
    all_stopwords = custom_stopwords.union(set(stopwords.words('english')))
    return [word for word in tokens if word not in all_stopwords]



6. Lematización

- Se descarga el recurso wordnet de nltk para usar el lematizador.
- La función lemmatize_tokens toma la lista de tokens y aplica la lematización a cada uno, devolviendo una lista con las palabras lematizadas.

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens: list) -> list:
    """
    Aplica lematización a los tokens.
    Args:
        tokens (list): Lista de tokens filtrados.
    Returns:
        list: Lista de tokens lematizados.
    """
    return [lemmatizer.lemmatize(token) for token in tokens]


7. Normalizacion

- Se utilizar un diccionario de normalización con términos comunes en noticias Reuters.

In [ ]:
def _load_normalization_dict(self) -> Dict[str, str]:
        """
        Carga el diccionario 
        """
        return {
            # Countries and Regions
            'uk': 'united kingdom',
            'britain': 'united kingdom',
            'great britain': 'united kingdom',
            'united kingdom': 'united kingdom',
            'england': 'united kingdom',
            'british': 'united kingdom',
            'us': 'united states',
            'usa': 'united states',
            'united states of america': 'united states',
            'united states': 'united states',
            'america': 'united states',
            'american': 'united states',
            
            # Organizational Entities
            'un': 'united nations',
            'united nations': 'united nations',
            'who': 'world health organization',
            'world health organization': 'world health organization',
            'imf': 'international monetary fund',
            'international monetary fund': 'international monetary fund',
            'wb': 'world bank',
            'world bank': 'world bank',
            'wto': 'world trade organization',
            'world trade organization': 'world trade organization',
            'nato': 'north atlantic treaty organization',
            'north atlantic treaty organization': 'north atlantic treaty organization',
            
            # Financial Markets
            'nyse': 'new york stock exchange',
            'nasdaq': 'nasdaq stock market',
            'djia': 'dow jones industrial average',
            'sp500': 'standard and poors 500',
            's&p': 'standard and poors 500',
            'ftse': 'financial times stock exchange',
            
            # Central Banks
            'fed': 'federal reserve',
            'federal reserve': 'federal reserve',
            'ecb': 'european central bank',
            'boe': 'bank of england',
            'pboc': 'peoples bank of china',
            'boj': 'bank of japan',
            
            # Economic Indicators
            'gdp': 'gross domestic product',
            'cpi': 'consumer price index',
            'ppi': 'producer price index',
            'pmi': 'purchasing managers index',
            'ism': 'institute supply management',
            'nfp': 'non farm payrolls',
            
            # Business Terms
            'ceo': 'chief executive officer',
            'cfo': 'chief financial officer',
            'coo': 'chief operating officer',
            'ipo': 'initial public offering',
            'ma': 'mergers acquisitions',
            'eps': 'earnings per share',
            'ebit': 'earnings before interest taxes',
            'ebitda': 'earnings before interest taxes depreciation amortization',
            
            # Commodities
            'wti': 'west texas intermediate',
            'brent': 'brent crude oil',
            'opec': 'organization petroleum exporting countries',
            'lng': 'liquefied natural gas',
            
            # Technology
            'ai': 'artificial intelligence',
            'ml': 'machine learning',
            'iot': 'internet of things',
            'saas': 'software as service',
            'paas': 'platform as service',
            'iaas': 'infrastructure as service',
            
            # Market Terms
            'ytd': 'year to date',
            'yoy': 'year over year',
            'qoq': 'quarter over quarter',
            'mom': 'month over month',
            'ttm': 'trailing twelve months',
            
            # Time Zones
            'est': 'eastern standard time',
            'edt': 'eastern daylight time',
            'gmt': 'greenwich mean time',
            'utc': 'coordinated universal time',
            
            # Common Industry Terms
            'capex': 'capital expenditure',
            'opex': 'operating expense',
            'r&d': 'research development',
            'roi': 'return on investment',
            'roic': 'return on invested capital',
            'roa': 'return on assets',
            'roe': 'return on equity'
        }




8. Guardado del Corpus Procesado

- Cada documento procesado se convierte en una fila del CSV con su nombre y contenido procesado.

In [ ]:
import pandas as pd

# Convertimos los textos procesados en un formato adecuado
processed_data = pd.DataFrame({
    'file_name': [doc['file_name'] for doc in documents],
    'processed_content': [' '.join(preprocess_document(doc['content'])) for doc in documents]
})

# Guardamos el archivo
processed_data.to_csv('processed_documents.csv', index=False)


9. Guardado de Estadísticas del Preprocesamiento
- Después de realizar todas las etapas de preprocesamiento se almacenan las estadísticas obtenidas para facilitar la evaluación posterior del sistema.

Estadísticas a guardar:

* Número de documentos procesados.
* Número de tokens procesados por documento.
* Número de tokens únicos (vocabulario total).
* Promedio de tokens por documento.

In [ ]:
import pandas as pd

def save_preprocessing_statistics(processed_docs: list, output_path: str):
    """
    Guarda las estadísticas del preprocesamiento de los documentos.
    Args:
        processed_docs (list): Lista de documentos procesados.
        output_path (str): Ruta donde se guardarán las estadísticas.
    """
    # Estadísticas de los documentos
    num_documents = len(processed_docs)
    total_tokens = sum([len(doc.split()) for doc in processed_docs])
    unique_tokens = len(set(" ".join(processed_docs).split()))
    avg_tokens_per_doc = total_tokens / num_documents if num_documents > 0 else 0

    stats = {
        'Número de Documentos': num_documents,
        'Tokens Totales': total_tokens,
        'Tokens Únicos': unique_tokens,
        'Promedio de Tokens por Documento': avg_tokens_per_doc
    }

    # Convertir estadísticas en un DataFrame
    stats_df = pd.DataFrame([stats])

    # Guardar estadísticas en archivo CSV
    stats_df.to_csv(output_path, index=False)
    print(f"Estadísticas del preprocesamiento guardadas en: {output_path}")


### 3. Representacióon de Datos en Espacio Vectorial


Esta fase consiste en transformar los documentos del corpus en matrices numéricas que los algoritmos pueden procesar para medir similitudes y realizar búsquedas eficaces. Además, se guarda esta información en un archivo CSV para su reutilización.

1. Carga de Datos Preprocesados

- Se cargan los datos preprocesados desde un archivo CSV
- Este archivo contiene el ID del documento, su título y el contenido procesado.



In [ ]:
# Cargar datos preprocesados
processed_data = pd.read_csv('reuters_preprocessed_clean.csv')

# Asegurar que las columnas tienen el formato correcto
processed_data['id'] = processed_data['id'].astype(str)
processed_data['body'] = processed_data['body'].astype(str)

# Extraer el texto procesado para vectorización
processed_texts = processed_data['body'].tolist()


2. Inicialización de Métodos de Vectorización

- Antes de aplicar cualquier técnica, se inicializan los métodos necesarios para generar las representaciones vectoriales.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim.downloader as api

# Configurar vectorizadores BoW y TF-IDF
bow_vectorizer = CountVectorizer(min_df=2, max_df=0.95)
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)

# Cargar modelo preentrenado Word2Vec
word2vec_model = api.load("word2vec-google-news-300")

# Inicializamos los vectorizadores
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
bow_vectorizer = CountVectorizer()
binary_vectorizer = CountVectorizer(binary=True)


3. Vectorización con TF-IDF

- Se genera la representación TF-IDF, asignando pesos a términos en función de su frecuencia relativa.

Cálculo de Estadísticas:
- Tamaño del vocabulario: Palabras únicas generadas.
- Valores de IDF: Medida de la importancia de los términos en el corpus.

In [ ]:
# Generar matriz TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_texts)

# Calcular estadísticas
tfidf_stats = {
    'matrix_shape': tfidf_matrix.shape,
    'vocabulary_size': len(tfidf_vectorizer.vocabulary_),
    'sparsity': 1.0 - (tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]))
}

# Generar estadísticas detalladas
tfidf_statistics = pd.DataFrame({
    'term': tfidf_vectorizer.get_feature_names_out(),
    'idf': tfidf_vectorizer.idf_
})
tfidf_statistics.to_csv('tfidf_statistics.csv', index=False)
print("Estadísticas TF-IDF guardadas en: tfidf_statistics.csv")



4. Vectorización con BoW 

- Se genera una matriz dispersa donde cada fila representa un documento y cada columna, un término del vocabulario.

Cálculo de Estadísticas:
* Tamaño de la matriz: Número de documentos x número de términos únicos.
* Sparsidad: Proporción de ceros en la matriz, indicando qué tan dispersa es.
* Tamaño del vocabulario: Número total de términos únicos.

In [ ]:
# Aplicar BoW
bow_matrix = bow_vectorizer.fit_transform(processed_texts)

# Generar matriz BoW
bow_matrix = bow_vectorizer.fit_transform(processed_texts)

# Calcular estadísticas
bow_stats = {
    'matrix_shape': bow_matrix.shape,
    'vocabulary_size': len(bow_vectorizer.vocabulary_),
    'sparsity': 1.0 - (bow_matrix.nnz / (bow_matrix.shape[0] * bow_matrix.shape[1]))
}

# Guardar estadísticas detalladas
import pandas as pd

bow_statistics = pd.DataFrame({
    'term': bow_vectorizer.get_feature_names_out(),
    'frequency': bow_matrix.sum(axis=0).tolist()[0]
})
bow_statistics.to_csv('bow_statistics.csv', index=False)
print("Estadísticas BoW guardadas en: bow_statistics.csv")



5. Vectorización con Word2Vec

- Word2Vec genera vectores denso sutilizando un modelo preentrenado que captura relaciones semánticas entre palabras.

Cálculo de Estadísticas:
* Cobertura del vocabulario: Proporción de palabras del corpus presentes en el modelo preentrenado.
* Dimensiones del vector: Longitud del vector generado para cada documento.

In [ ]:
import numpy as np
from gensim.utils import simple_preprocess

# Generar vectores promedio para cada documento
vector_size = word2vec_model.vector_size
doc_vectors = []
words_found = 0
total_words = 0

for doc in processed_texts:
    words = simple_preprocess(doc)
    total_words += len(words)
    
    vec = np.zeros(vector_size)
    count = 0
    
    for word in words:
        if word in word2vec_model:
            vec += word2vec_model[word]
            count += 1
            words_found += 1
    
    if count > 0:
        vec /= count
    doc_vectors.append(vec)

# Matriz resultante
word2vec_matrix = np.vstack(doc_vectors)

# Calcular estadísticas
word2vec_stats = {
    'matrix_shape': word2vec_matrix.shape,
    'vocabulary_coverage': (words_found / total_words) * 100
}

# Guardar estadísticas
word2vec_stats_df = pd.DataFrame([word2vec_stats])
word2vec_stats_df.to_csv('word2vec_statistics.csv', index=False)
print("Estadísticas Word2Vec guardadas en: word2vec_statistics.csv")



6. Consolidación de Estadísticas de Vectorización

Todas las estadísticas generadas se consolidan en un único archivo para comparar los métodos.


In [ ]:
# Consolidar estadísticas
vectorization_summary = pd.DataFrame([
    {'Método': 'BoW', 'Términos Únicos': len(bow_statistics), 'Archivo': 'bow_statistics.csv'},
    {'Método': 'TF-IDF', 'Términos Únicos': len(tfidf_statistics), 'Archivo': 'tfidf_statistics.csv'},
    {'Método': 'Word2Vec', 'Dimensiones del Vector': vector_size, 'Cobertura del Vocabulario (%)': vocab_coverage, 'Archivo': 'word2vec_statistics.csv'}
])

# Guardar resumen consolidado
vectorization_summary.to_csv('vectorization_summary.csv', index=False)
print("Resumen de vectorización guardado en: vectorization_summary.csv")


7. Evaluación de los Métodos de Vectorización

Para evaluar los métodos, se compararon métricas clave como precisión, recall y tiempo de cálculo. Estas métricas se calculan tras realizar búsquedas en el corpus usando cada técnica.

In [ ]:
def evaluate_vectorization(vectorizer, queries, corpus_matrix):
    """
    Evalúa un método de vectorización utilizando métricas de recuperación.
    Args:
        vectorizer: Vectorizador (e.g., TF-IDF, BoW, Word2Vec).
        queries: Lista de consultas de prueba.
        corpus_matrix: Matriz de documentos vectorizados.
    Returns:
        Dict: Métricas de evaluación.
    """
    metrics = {'precision': [], 'recall': [], 'f1_score': []}
    for query in queries:
        query_vector = vectorizer.transform([query])
        results = cosine_similarity(query_vector, corpus_matrix)
        # Aquí se calcularían las métricas con base en los resultados obtenidos
        precision, recall, f1 = calculate_metrics(results)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1_score'].append(f1)
    
    # Promedio de las métricas
    avg_metrics = {k: np.mean(v) for k, v in metrics.items()}
    return avg_metrics


5. Selección de la Representación

- Una vez generadas las matrices, se selecciona el método vectorial a utilizar según el objetivo.

In [ ]:
selected_matrix = tfidf_matrix  # Cambiar a bow_matrix o binary_matrix según el caso


### 4. Indexación

El índice invertido organiza las palabras del corpus junto con los documentos en los que aparecen. Esto permite búsquedas rápidas al localizar documentos relevantes directamente a partir de los términos de consulta.

1. Estructura del índice

El índice es implementado como un diccionario donde cada término tiene como valor una lista de identificadores de documentos.

- La clase InvertedIndex usa un diccionario de Python para almacenar el índice, donde cada término apunta a un conjunto de documentos en los que aparece.
- El método add_document agrega los términos de un documento a este índice.

In [ ]:
import csv
from collections import defaultdict
import pandas as pd

class InvertedIndex:
    def __init__(self):
        # Usamos defaultdict para facilitar la creación de listas automáticamente
        self.index = defaultdict(set)  # Mapea términos a un conjunto de doc_ids

    def add_document(self, doc_id: str, terms: list):
        """
        Añade los términos de un documento al índice invertido.
        
        Args:
            doc_id (str): Identificador único del documento.
            terms (list): Lista de términos del documento.
        """
        for term in terms:
            self.index[term].add(doc_id)


2. Carga del Corpus y Construcción del Índice Invertido

Se recorren todos los documentos preprocesados para añadirlos al índice.

- La función build_inverted_index recorre una lista de documentos, procesando su contenido y añadiendo los términos al índice invertido usando la clase InvertedIndex.

In [ ]:
def build_inverted_index(processed_data: pd.DataFrame) -> InvertedIndex:
    """
    Construye el índice invertido a partir de los documentos procesados.

    Args:
        processed_data (pd.DataFrame): DataFrame que contiene los documentos preprocesados.
        
    Returns:
        InvertedIndex: El índice invertido que contiene los términos mapeados a los documentos.
    """
    index = InvertedIndex()
    
    for index, row in processed_data.iterrows():
        doc_id = row['id']
        content = row['body']
        terms = preprocess_document(content)  # Utilizamos el preprocesamiento de los textos
        index.add_document(doc_id, terms)
    
    return index


3. Guardado del Índice Invertido

- El índice invertido debe guardarse en un archivo CSV para su posterior recuperación y uso en la búsqueda.

In [ ]:
def save_inverted_index(index: InvertedIndex, output_file: str):
    """
    Guarda el índice invertido en un archivo CSV.
    
    Args:
        index (InvertedIndex): El índice invertido a guardar.
        output_file (str): Ruta al archivo donde se guardará el índice.
    """
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['term', 'doc_ids'])  # Cabeceras
        for term, doc_ids in index.index.items():
            writer.writerow([term, " ".join(doc_ids)])  # Convertimos doc_ids a string
    print(f"Índice invertido guardado en: {output_file}")


### 5. Mecanismo de búsqueda 

La búsqueda se realiza comparando la consulta del usuario con los documentos preprocesados y vectorizados. La similitud del coseno es la métrica utilizada para determinar qué tan similares son dos vectores en el espacio de características.

- Flujo del Motor de Búsqueda:

1.Ingreso de la Consulta → 2. Preprocesamiento → 3. Vectorización → 4. Cálculo de Similitud → 5. Ranking → 6. Recuperación → 7. Salida.

1. Procesamiento de la Consulta

- La consulta del usuario se preprocesa para convertirla en un formato que pueda ser comparado con los documentos del corpus.
-  El procesamiento de la consulta sigue los mismos pasos que se aplicaron a los documentos en el corpus.


In [ ]:
def preprocess_query(query: str) -> list:
    """
    Preprocesa la consulta del usuario (tokenización, eliminación de stopwords, lematización).
    
    Args:
        query (str): La consulta introducida por el usuario.
    
    Returns:
        list: Lista de tokens procesados.
    """
    query_tokens = preprocess_document(query)  # Usamos el preprocesamiento ya definido
    return query_tokens



2. Transformación de la Consulta en Vector
- Una vez procesada la consulta, se transforma en un vector utilizando el mismo vectorizador que se usó para los documentos.


In [ ]:
# Preprocesamos la consulta
query_tokens = preprocess_query(query)  # Tokenización y limpieza
query_vec = vectorizer.transform([' '.join(query_tokens)])  # Transformar la consulta en vector utilizando el vectorizador


3. Cálculo de la Similitud  Coseno entre la Consulta y los Documentos

- Después de que la consulta se ha convertido en un vector, el siguiente paso es calcular su similitud con cada uno de los documentos en el corpus. Utilizamos la similitud coseno, que mide el ángulo entre los vectores de la consulta y los documentos. 

- Utilizamos la función de scikit-learn para calcular la similitud coseno entre el vector de la consulta y la matriz de documentos.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(query_vector, document_matrix):
    """
    Calcula la similitud coseno entre la consulta y los documentos.
    
    Args:
        query_vector (sparse matrix): Vector de la consulta procesada.
        document_matrix (sparse matrix): Matriz de los documentos preprocesados (BoW o TF-IDF).
    
    Returns:
        np.ndarray: Similitudes entre la consulta y los documentos.
    """
    return cosine_similarity(query_vector, document_matrix)


4. Ranking de los Resultados

- Una vez calculadas las similitudes, necesitamos ordenar los documentos de acuerdo con su relevancia.
- El ranking se hace ordenando las similitudes de mayor a menor, para devolver los documentos más relevantes en la parte superior.

In [ ]:
import numpy as np

def rank_documents(similarities, top_k=5):
    """
    Ordena los documentos según su similitud con la consulta y devuelve los primeros `k` documentos.
    
    Args:
        similarities (np.ndarray): Array de similitudes entre la consulta y los documentos.
        top_k (int): Número máximo de resultados a retornar.
    
    Returns:
        list: Lista de índices de documentos ordenados por relevancia.
    """
    sorted_indices = np.argsort(similarities)[::-1][:top_k]  # Ordenar y seleccionar top_k
    return sorted_indices


5. Recuperación de los Documentos Relevantes

- Después de ordenar los documentos por similitud, se recuperan los top_k documentos más relevantes. Esta función extrae la información de los documentos (título, cuerpo y score) para mostrarlos al usuario.



In [ ]:
def retrieve_documents(sorted_indices, corpus_df):
    """
    Recupera los documentos relevantes del corpus basándose en los índices ordenados.
    
    Args:
        sorted_indices (list): Índices de los documentos ordenados por similitud.
        corpus_df (pd.DataFrame): DataFrame con el corpus de documentos.
    
    Returns:
        list: Lista de documentos relevantes.
    """
    results = []
    for idx in sorted_indices:
        doc_info = corpus_df.iloc[idx]
        results.append({
            'id': doc_info['id'],
            'title': doc_info['title'],
            'body': doc_info['body'][:200] + '...',  # Primeros 200 caracteres
            'score': float(similarities[idx])
        })
    return results


6.  Motor de Búsqueda Completo

El método principal del motor de búsqueda:
1. Preprocesa la consulta.
2. La convierte en un vector utilizando el vectorizador.
3. Calcula las similitudes con la matriz de documentos.
4. Ordena los documentos por relevancia.
5. Recupera y devuelve los top_k documentos más relevantes.

In [ ]:
class SearchEngine:
    def __init__(self, corpus_df, vectorizer):
        self.corpus_df = corpus_df
        self.vectorizer = vectorizer
        self.document_matrix = vectorizer.transform(corpus_df['body'])
    
    def search(self, query, top_k=5):
        # Preprocesamos la consulta
        query_tokens = preprocess_query(query)  # Preprocesamiento de la consulta
        query_vec = self.vectorizer.transform([' '.join(query_tokens)])  # Transformar la consulta en vector

        # Calculamos la similitud
        similarities = calculate_cosine_similarity(query_vec, self.document_matrix)  # Similitud coseno
        sorted_indices = rank_documents(similarities.flatten(), top_k)  # Ranking de resultados

        # Recuperamos los documentos más relevantes
        return retrieve_documents(sorted_indices, self.corpus_df)


### 6.Evaluación del Sistema

Se utilizan métricas estándar como precisión, recall, F1-score y mean average precision (MAP) para evaluar el desempeño del sistema 

1. Implementación de la Evaluación

- Para evaluar el sistema, se utiliza un conjunto de consultas de prueba. Para cada consulta, se calculan las métricas de precisión, recall, F1-score, y MAP.
- Posteriormente, se realiza una evaluación global del sistema utilizando estas métricas.


In [ ]:
def evaluate_query(query: str, relevant_docs: str, k: int = 10) -> dict:
    """
    Evalúa una consulta individual.
    
    Args:
        query (str): La consulta del usuario.
        relevant_docs (str): Documentos relevantes para la consulta.
        k (int): Número de resultados a considerar.
    
    Returns:
        dict: Diccionario con métricas de la consulta.
    """
    relevant_set = set(relevant_docs.split())  # Convertimos los documentos relevantes en un set
    search_results = search_engine.search(query, num_results=k)  # Realizamos la búsqueda
    retrieved_set = {str(result['id']) for result in search_results}  # Convertimos los documentos recuperados en un set
    
    # Cálculo de precisión, recall y F1-score
    relevant_retrieved = retrieved_set & relevant_set
    precision = len(relevant_retrieved) / len(retrieved_set) if retrieved_set else 0
    recall = len(relevant_retrieved) / len(relevant_set) if relevant_set else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Cálculo de MAP (Average Precision)
    ap = calculate_average_precision(search_results, relevant_set)
    
    # Cálculo de nDCG (Normalized Discounted Cumulative Gain)
    dcg = calculate_dcg(search_results, relevant_set, k)
    idcg = calculate_idcg(len(relevant_set), k)
    ndcg = dcg / idcg if idcg > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'average_precision': ap,
        'ndcg': ndcg,
        'retrieved_count': len(retrieved_set),
        'relevant_count': len(relevant_set),
        'relevant_retrieved_count': len(relevant_retrieved)
    }


2. Cálculo del MAP

- El método calculate_average_precision calcula la precisión promedio para una consulta al sumar las precisiones en cada posición del ranking de los documentos relevantes.

In [ ]:
def calculate_average_precision(results: list, relevant_docs: set) -> float:
    """
    Calcula la precisión promedio (AP) para una consulta.
    
    Args:
        results (list): Resultados de la consulta.
        relevant_docs (set): Conjunto de documentos relevantes para la consulta.
    
    Returns:
        float: La precisión promedio.
    """
    precision_sum = 0
    relevant_found = 0
    for i, result in enumerate(results, 1):
        if str(result['id']) in relevant_docs:
            relevant_found += 1
            precision_at_k = relevant_found / i
            precision_sum += precision_at_k
    
    return precision_sum / len(relevant_docs) if relevant_docs else 0


3. Cálculo de nDCG

- El método calculate_dcg calcula el DCG basado en la relevancia de los documentos. El DCG penaliza los documentos relevantes que están en posiciones más bajas de la lista de resultados, lo que refleja que es más importante que los documentos relevantes estén en las primeras posiciones.

In [ ]:
def calculate_dcg(results: list, relevant_docs: set, k: int) -> float:
    """
    Calcula Discounted Cumulative Gain (DCG) para los primeros `k` resultados.
    
    Args:
        results (list): Resultados de la búsqueda.
        relevant_docs (set): Conjunto de documentos relevantes.
        k (int): Número de resultados a considerar.
    
    Returns:
        float: El valor de DCG.
    """
    dcg = 0
    for i, result in enumerate(results[:k], 1):
        rel = 1 if str(result['id']) in relevant_docs else 0
        dcg += rel / np.log2(i + 1)
    return dcg


4. Evaluación del Sistema Completo

- Se evalúa el rendimiento del sistema en su totalidad utilizando un conjunto de consultas de prueba.


In [ ]:
def evaluate_system(test_queries: pd.DataFrame, k_values: list = [5, 10, 20]) -> dict:
    """
    Evalúa el sistema completo para diferentes valores de k (top-k resultados).
    
    Args:
        test_queries (pd.DataFrame): Conjunto de consultas de prueba.
        k_values (list): Lista de valores de k para evaluar.
    
    Returns:
        dict: Resultados de la evaluación con las métricas para cada k.
    """
    results = {}
    
    for k in k_values:
        query_results = []
        for _, row in tqdm(test_queries.iterrows(), total=len(test_queries)):
            eval_result = evaluate_query(row['query'], row['relevant_docs'], k)
            eval_result['query'] = row['query']
            query_results.append(eval_result)
        
        avg_metrics = {
            'avg_precision': np.mean([r['precision'] for r in query_results]),
            'avg_recall': np.mean([r['recall'] for r in query_results]),
            'avg_f1': np.mean([r['f1_score'] for r in query_results]),
            'mean_ap': np.mean([r['average_precision'] for r in query_results]),
            'avg_ndcg': np.mean([r['ndcg'] for r in query_results])
        }
        
        results[k] = {
            'query_results': query_results,
            'average_metrics': avg_metrics
        }
        
    return results


5. Guardado de los Resultados de Evaluación

- Una vez que se calculan las métricas de evaluación para cada consulta y se obtiene un resumen global del sistema, es importante guardar estos resultados de forma organizada.

- Los resultados los guardamos en dos formatos diferntes en un archivo JSON y un CSV



#### Archivo JSON

In [ ]:
import json
from pathlib import Path
from datetime import datetime

def save_detailed_results(results: dict, output_dir: str = 'evaluation_results'):
    """
    Guarda los resultados detallados de la evaluación en un archivo JSON.
    
    Args:
        results (dict): Los resultados de la evaluación por consulta.
        output_dir (str): Directorio donde se guardarán los resultados.
    """
    # Crear el directorio de salida si no existe
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Generar un timestamp para nombrar el archivo
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Guardar los resultados detallados en formato JSON
    with open(f'{output_dir}/detailed_results_{timestamp}.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"Resultados detallados guardados en: {output_dir}/detailed_results_{timestamp}.json")


#### Archivo CSV 

In [ ]:
import pandas as pd

def save_summary_results(results: dict, output_dir: str = 'evaluation_results'):
    """
    Guarda los resultados agregados de la evaluación en un archivo CSV.
    
    Args:
        results (dict): Los resultados de la evaluación por consulta.
        output_dir (str): Directorio donde se guardarán los resultados.
    """
    # Crear el directorio de salida si no existe
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Convertir los resultados agregados en un DataFrame
    summary_data = []
    for k, result in results.items():
        summary_data.append({
            'k': k,
            'avg_precision': result['average_metrics']['avg_precision'],
            'avg_recall': result['average_metrics']['avg_recall'],
            'avg_f1': result['average_metrics']['avg_f1'],
            'mean_ap': result['average_metrics']['mean_ap'],
            'avg_ndcg': result['average_metrics']['avg_ndcg']
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Guardar los resultados agregados en un archivo CSV
    summary_df.to_csv(f'{output_dir}/evaluation_summary.csv', index=False)
    print(f"Resultados agregados guardados en: {output_dir}/evaluation_summary.csv")
